**NOTE:** Due to the high computational cost of SHAP, the runtime of this script can be long.

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [3]:
import shap, datasets, pickle, datetime, copy, os

import pandas as pd
import numpy as np
import scipy as sp
import tensorflow as tf

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from shap import Explanation
from shap.plots._utils import convert_ordering
from shap.utils import ordinal_str
from collections import defaultdict

2023-05-10 23:58:53.602737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1. Set the model to use

**Uncomment one of the lines below to set the model.**

In [4]:
MODEL_NAME = "bert-base-uncased"
#MODEL_NAME = "roberta-base"
#MODEL_NAME = "xlnet-base-cased"

# 2. Load dataset and set which model to use

In [5]:
df = pd.read_csv("../data/figlang_all.tsv", sep="\t", encoding="utf-8")
print(df.shape)
print(df["label"].value_counts())
df.head()

(10848, 4)
0    6506
1    2212
2     884
3     625
4     621
Name: label, dtype: int64


,text,label,label_binary,source
0,I can't believe my ex didn't pay his car note ...,0,0,Sarcasm_premise
1,But then the paper would not find out about yo...,0,0,Idiom_premise
2,Last week my kid said some really mean things ...,0,0,CreativeParaphrase_premise
3,"The gravy was so fatty, it made the meat taste...",0,0,Metaphor_premise
4,He pulls a giant disc out and flashes it like ...,3,1,Simile_hypothesis


In [6]:
df_multi = df[df["label"] != 0].reset_index()
print(df_multi.shape)
print(df_multi["label"].value_counts())
df_multi.head()

(4342, 5)
1    2212
2     884
3     625
4     621
Name: label, dtype: int64


,index,text,label,label_binary,source
0,4,He pulls a giant disc out and flashes it like ...,3,1,Simile_hypothesis
1,9,Some bright young thing had gotten ahold of a ...,2,1,Idiom_hypothesis
2,13,"“I might be mistaken, but Sean's father looked...",2,1,Idiom_hypothesis
3,14,Her movements like a strange strip tease .,3,1,Simile_hypothesis
4,16,"I had to leave my childhood home, and am grate...",1,1,Sarcasm_hypothesis


In [7]:
dataset = datasets.Dataset.from_pandas(df_multi)
print(len(dataset))
dataset

4342


Dataset({
    features: ['index', 'text', 'label', 'label_binary', 'source'],
    num_rows: 4342
})

# 3. Load model

In [8]:
model_path_dict = {
    "bert-base-uncased": "../results/models/bert-base-uncased-figlang",
    "roberta-base": "../results/models/roberta-base-figlang",    
    "xlnet-base-cased": "../results/models/xlnet-base-cased-figlang",
}

In [9]:
output_dir = model_path_dict[MODEL_NAME]
print(output_dir)

../results/models/bert-base-uncased-figlang


In [10]:
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForSequenceClassification.from_pretrained(output_dir)
model.cuda()
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

**Run a tokenization to get the maximal sequence length:**

In [11]:
MAX_LEN = 0

for text in dataset["text"]:
    input_ids = tokenizer.encode(text, 
                                 add_special_tokens=True)
    MAX_LEN = max(MAX_LEN, len(input_ids))

print('Max sentence length: ', MAX_LEN)

Max sentence length:  53


# 4. Compute SHAP

In [12]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=MAX_LEN, truncation=True) for v in x]).to(device)#.cuda()
    attention_mask = (tv!=0).type(torch.int64).cuda() 
    outputs = model(tv, attention_mask=attention_mask)[0].detach().cpu().numpy()
    #outputs = model(tv)[0].detach().cpu().numpy()
    
    return outputs

In [13]:
explainer = shap.Explainer(f, tokenizer)

In [14]:
shap_values = explainer(dataset["text"])

Partition explainer: 4343it [49:47,  1.45it/s]                           


**Save SHAP values:**

In [15]:
with open('../results/shap/shap_values_' + output_dir.split("/")[-1] + '.pkl', 'wb') as out_value:
    pickle.dump(shap_values, out_value, pickle.HIGHEST_PROTOCOL)

**Save explainer:**

In [17]:
with open("../results/shap/shap_explainer_" + output_dir.split("/")[-1] + ".sav", "wb") as out:
    explainer.save(out)

# 5. Write out results

In [18]:
model_shap_dict = {
    "bert-base-uncased": "../results/shap/shap_values_bert-base-uncased-figlang.pkl",
    "roberta-base": "../results/shap/shap_values_roberta-base-figlang",
    "xlnet-base-cased": "../results/shap/shap_values_xlnet-base-cased-figlang.pkl",
}

In [19]:
label_dict = {
    0: "sarcasm", 
    1: "idiom",
    2: "simile",
    3: "metaphor"
}

In [20]:
def get_topk_features_by_class(shap_values, label, top_k):    
    cohorts = {"": shap_values[:,:,label]}
    cohort_exps = list(cohorts.values())
    #cohort_exps[0] = cohort_exps[0].abs.mean(0)
    cohort_exps[0] = cohort_exps[0].mean(0)
       
    features = cohort_exps[0].data
    feature_names = cohort_exps[0].feature_names
    
    values = np.array([cohort_exps[i].values for i in range(len(cohort_exps))])
    
    feature_order = np.argsort(np.mean([np.argsort(convert_ordering(Explanation.argsort.flip, Explanation(values[i]))) for i in range(values.shape[0])], 0))
    feature_inds = feature_order[:top_k]
    
    top_k_features = {}
    for i in feature_inds:
        #top_k_features.append(feature_names[i])
        top_k_features[feature_names[i]] = values[0][i]

    return top_k_features

In [21]:
def get_corpus_feature_count(shap_values, label):
    docs = shap_values.data
    
    corpus_feature_count = defaultdict(int)
    for doc in docs:
        for i in doc:
            corpus_feature_count[i.strip()] += 1
    
    return corpus_feature_count

In [22]:
def get_feature_count_by_label(doc_df, shap_values, label):
    indice = list(doc_df.loc[doc_df["label"] == label, ].index)
    docs = np.array(shap_values.data)[indice]
    
    feature_count_per_label = defaultdict(int)
    for doc in docs:
        for i in doc:
            feature_count_per_label[i.strip()] += 1
    
    return feature_count_per_label

In [23]:
shap_path = model_shap_dict[MODEL_NAME]
shap_values = pickle.load(open(shap_path, 'rb'))

In [24]:
doc_df = df_multi

for i in range(len(label_dict)):
    print("Processing class {}".format(i))
    top_k_features = get_topk_features_by_class(shap_values=shap_values, label=i, top_k=50)
    feature_count_corpus = get_corpus_feature_count(shap_values=shap_values, label=i)
    feature_count_per_label = get_feature_count_by_label(doc_df=doc_df, shap_values=shap_values, label=i)
    
    freq_by_label = []
    freq_corpus = []

    for feature in top_k_features.keys():
        freq_by_label.append(feature_count_per_label[feature])
        freq_corpus.append(feature_count_corpus[feature])
    
    label_corpus_freq_ratio = [i / j for i, j in zip(freq_by_label, freq_corpus)]
    
    df_result = pd.DataFrame({"feature": top_k_features.keys(),
                              "shap": top_k_features.values(),
                              "freq_by_label": freq_by_label,
                              "freq_corpus": freq_corpus,
                              "label_corpus_freq_ratio": label_corpus_freq_ratio,
                             })
    
    
    df_result.to_csv("../results/shap/top_50_features_blackbox/" + "_".join(shap_path.split("/")[-1].split(".")[0].split("_")[2:]) 
                     + "_" + label_dict[i] + ".tsv", 
                     sep="\t", encoding="utf-8", index=False)
    
print("Done.")

Processing class 0


Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Processing class 1
Processing class 2
Processing class 3
Done.
